In [ ]:
!pip install datasets transformers torch datasets evaluate allennlp
!pip install --upgrade huggingface-hub==0.24.0
from huggingface_hub import split_torch_state_dict_into_shards
!pip uninstall -y torch transformers
!pip install allennlp==2.10.1
!pip install evaluate datasets transformers
!pip install pytorch_pretrained_bert
!pip install datasets
from google.colab import drive
drive.mount('/content/drive')
import os,sys, allennlp
import knowbert
import model
os.chdir('/content/drive/My Drive/Gatech courses/Fall 2024 Courses/Natural Language Processing/NLP Project/Code/knowbert')
#os.chdir('/content/drive/MyDrive/Uni/Semesters/2024 Fall/CS 7650 (NLP)/NLP Project')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of cached-path to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 4.8 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of cached-path to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import nltk
from typing import Union, List
from datasets import Dataset, DatasetDict
from tqdm import tqdm
from google.colab import drive
import requests
import tarfile
import tempfile
import torch
from typing import Union
import logging
import shutil
from pathlib import Path
#from allennlp.models.archival import load_archive
# from allennlp.common import Params
# from allennlp.data import Instance, Vocabulary
# from allennlp.data.data_loaders import MultiProcessDataLoader  # this is the new import for DataIterator
# from allennlp.common.file_utils import cached_path

#from kb.knowbert_utils import KnowBertBatchifier
from typing import List, Tuple, Dict, Optional
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class KnowBERTProcessor:
    def __init__(self, root_path: str, batch_size: int = 32):
        """
        Initialize KnowBERT processor with paths and configurations

        Args:
            root_path: Base path for all data files
            batch_size: Batch size for processing
        """
        self.root_path = root_path
        self.batch_size = batch_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None
        self.batcher = None

        # Download NLTK data
        nltk.download('punkt', quiet=True)

        logger.info(f"Initialized KnowBERTProcessor using device: {self.device}")

    def load_datasets(self, isear_path: str, conceptnet_path: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Load ISEAR and ConceptNet datasets

        Args:
            isear_path: Path to ISEAR dataset
            conceptnet_path: Path to ConceptNet dataset

        Returns:
            Tuple of (isear_df, conceptnet_df)
        """
        full_isear_path = os.path.join(self.root_path, isear_path)
        full_conceptnet_path = os.path.join(self.root_path, conceptnet_path)

        # Verify paths exist
        if not os.path.exists(full_isear_path):
            raise FileNotFoundError(f"ISEAR dataset not found at: {full_isear_path}")
        if not os.path.exists(full_conceptnet_path):
            raise FileNotFoundError(f"ConceptNet dataset not found at: {full_conceptnet_path}")

        # Load datasets
        isear_df = pd.read_csv(full_isear_path)
        conceptnet_df = pd.read_csv( # returns an iterator
            full_conceptnet_path,
            sep='\t',
            usecols=[2, 3, 4],
            names=['start', 'rel', 'end'],
            compression='gzip',
            engine='python',
            chunksize=10000
        )
        print(f"Loaded ISEAR dataset with {len(isear_df)} rows")
        print(f"Each loaded ConceptNet chunk {len(next(conceptnet_df))} rows")
        return isear_df, conceptnet_df

    def download_model(url: str, save_path: str = 'model.tar.gz') -> str:
      """Download the model archive from a URL if not already downloaded."""
      if not os.path.exists(save_path):
          response = requests.get(url, stream=True)
          with open(save_path, 'wb') as file:
              for chunk in response.iter_content(chunk_size=1024):
                  if chunk:
                      file.write(chunk)
      return save_path

    def load_archive_helper(
      self,
      archive_file: Union[str, os.PathLike],
      cuda_device: int = -1,
      weights_file: str = None
  ) -> torch.nn.Module:
      """
      This function handles the loading of a PyTorch model from a tar.gz archive file that contains model weights (and possibly other associated files like configuration files

      Parameters:
      - archive_file (Union[str, os.PathLike]): Path to the archive file.
      - cuda_device (int): Device to load the model on. `-1` for CPU, >=0 for GPU device.
      - weights_file (str, optional): Specific weights file to use from the archive if specified.

      Returns:
      - model (torch.nn.Module): Loaded PyTorch model.
      """
      if not os.path.exists(archive_file):
          raise FileNotFoundError(f"Archive file not found: {archive_file}")

      tempdir = None
      try:
          tempdir = tempfile.mkdtemp()
          print(f"Extracting archive file {archive_file} to temporary directory {tempdir}")
          with tarfile.open(archive_file, "r:gz") as archive:
              archive.extractall(tempdir)

          weights_path = weights_file or os.path.join(tempdir, 'weights.th')
          if not os.path.exists(weights_path):
              raise FileNotFoundError(f"Expected weights file not found in archive: {weights_path}")

          device = torch.device("cuda" if cuda_device >= 0 and torch.cuda.is_available() else "cpu")
          model = torch.load(weights_path, map_location=device)
          model.eval()
          print(f"Successfully loaded model from {weights_path} onto {device}")
          return model
      finally:
          if tempdir is not None:
              print(f"Removing temporary directory {tempdir}")
              shutil.rmtree(tempdir, ignore_errors=True)

    def load_archive(self,archive_file: Union[str, os.PathLike], cuda_device: int = -1) -> torch.nn.Module:
      """
      Load a PyTorch knowbert model from a `tar.gz` archive file.

      Parameters:
      - archive_file (Union[str, os.PathLike]): Path to the archive file.
      - cuda_device (int): Device to load the model on. `-1` for CPU, >=0 for GPU device.

      Returns:
      - model (torch.nn.Module): Loaded PyTorch model.
      """
      if archive_file.startswith("http://") or archive_file.startswith("https://"):
        local_archive_file = 'model_download.tar.gz'
        if not os.path.exists(local_archive_file):
            response = requests.get(archive_file, stream=True)
            with open(local_archive_file, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
      else:
          local_archive_file = archive_file

      tempdir = None
      try:
          tempdir = tempfile.mkdtemp()
          print(f"Extracting archive file {local_archive_file} to temporary directory {tempdir}")
          with tarfile.open(local_archive_file, "r:gz") as archive:
              archive.extractall(tempdir)
              extracted_files = os.listdir(tempdir)
              print(f"Extracted files: {extracted_files}")

          weights_file = os.path.join(tempdir, 'weights.th')
          if not os.path.exists(weights_file):
              raise FileNotFoundError(f"Expected weights file not found in archive: {weights_file}")

          config_path = os.path.join(tempdir, 'config.json')
          if not os.path.exists(config_path):
              raise FileNotFoundError(f"Expected config file not found in archive: {config_path}")

          archive = self.load_archive_helper(
              local_archive_file,
              cuda_device,
              weights_file
          )

          return archive
      finally:
          if tempdir is not None:
              logger.info(f"Removing temporary directory {tempdir}")
              shutil.rmtree(tempdir, ignore_errors=True)

    def initialize_knowbert(self, model_url: str = 'https://allennlp.s3-us-west-2.amazonaws.com/knowbert/models/knowbert_wiki_wordnet_model.tar.gz'):
        """
        Initialize pre-trained KnowBERT model hosted on AllenNLP's S3 bucket, pretrained on Wikipedia and WordNet knowledge and batchifier

        Args:
            model_url: URL for the KnowBERT model
        """
        try:
            archive = self.load_archive(model_url)
            self.model = archive.model.to(self.device)
            self.batcher = KnowBertBatchifier(model_url)
            self.model.eval()
            logger.info("Successfully initialized KnowBERT model")
        except Exception as e:
            logger.error(f"Error initializing KnowBERT: {str(e)}")
            raise

    def get_conceptnet_triples(self, entity: str, conceptnet_df: pd.DataFrame) -> List[Tuple[str, str]]:
        """
        Get relevant triples from ConceptNet for a given entity

        Args:
            entity: Entity to look up
            conceptnet_df: ConceptNet DataFrame

        Returns:
            List of (relation, end_entity) tuples
        """
        entity = entity.lower().replace(" ", "_")
        triples = conceptnet_df[conceptnet_df['start'].str.endswith(f"/c/en/{entity}")][['rel', 'end']]
        return [(row['rel'].split('/')[-1], row['end'].split('/')[-1])
                for _, row in triples.iterrows()]

    def augment_sentence(self, sentence: str, conceptnet_df: pd.DataFrame, max_length: int = 512) -> str:
        """
        Augment a sentence with ConceptNet knowledge, by adding related conceptual knowledge to it.

        Args:
            sentence: Input sentence
            conceptnet_df: ConceptNet DataFrame
            max_length: Maximum length of augmented sentence

        Returns:
            Augmented sentence
        """
        entities = word_tokenize(sentence)
        augmented_sentence = sentence

        for entity in entities:
            triples = self.get_conceptnet_triples(entity, conceptnet_df)
            for rel, end in triples:
                augmented_sentence += f" {rel} {end}"

        return augmented_sentence[:max_length]

    def process_batch(self, sentences: List[str]) -> np.ndarray:
        """
        Process a batch of sentences through KnowBERT,  converts a group of sentences into embeddings using the KnowBERT model

        Args:
            sentences: List of sentences to process

        Returns:
            NumPy array of embeddings
        """
        if self.model is None or self.batcher is None:
            raise ValueError("KnowBERT model not initialized. Call initialize_knowbert first.")

        with torch.no_grad():
            batch = self.batcher.batchify(sentences)
            model_output = self.model(**batch.to(self.device))
            embeddings = model_output['contextual_embeddings']
            return embeddings.cpu().numpy()

    def process_dataset(self, df: pd.DataFrame, conceptnet_df: pd.DataFrame,
                       max_length: int = 512, save_path: Optional[str] = None) -> np.ndarray:
        """
        Process entire dataset with batching

        Args:
            df: Input DataFrame
            conceptnet_df: ConceptNet DataFrame
            max_length: Maximum sentence length
            save_path: Optional path to save embeddings

        Returns:
            NumPy array of embeddings
        """
        # Augment sentences
        logger.info("Augmenting sentences with ConceptNet knowledge...")
        augmented_sentences = [
            self.augment_sentence(sentence, conceptnet_df, max_length)
            for sentence in tqdm(df['content'], desc="Augmenting sentences")
        ]

        # Process in batches
        embeddings_list = []
        for i in tqdm(range(0, len(augmented_sentences), self.batch_size), desc="Processing batches"):
            batch_sentences = augmented_sentences[i:i + self.batch_size]
            batch_embeddings = self.process_batch(batch_sentences)
            embeddings_list.append(batch_embeddings)

        # Combine embeddings
        all_embeddings = np.vstack(embeddings_list)
        logger.info(f"Generated embeddings with shape: {all_embeddings.shape}")

        # Save if path provided
        if save_path:
            full_save_path = os.path.join(self.root_path, save_path)
            np.save(full_save_path, all_embeddings)
            logger.info(f"Saved embeddings to: {full_save_path}")
        return all_embeddings


class KnowBertBatchifier:
    """
    Takes a list of sentence strings and returns a tensor dict usable with
    a KnowBert model
    """
    def __init__(self, model_archive, batch_size=32,
                       masking_strategy=None,
                       wordnet_entity_file=None, vocab_dir=None):

        config = _extract_config_from_archive(cached_path(model_archive))

        candidate_generator_params = _find_key(
            config['dataset_reader'].as_dict(), 'tokenizer_and_candidate_generator'
        )

        if wordnet_entity_file is not None:
            candidate_generator_params['entity_candidate_generators']['wordnet']['entity_file'] = wordnet_entity_file

        self.tokenizer_and_candidate_generator = TokenizerAndCandidateGenerator.\
                from_params(Params(candidate_generator_params))
        self.tokenizer_and_candidate_generator.whitespace_tokenize = False

        assert masking_strategy is None or masking_strategy == 'full_mask'
        self.masking_strategy = masking_strategy

        if vocab_dir is not None:
            vocab_params = Params({"directory_path": vocab_dir})
        else:
            vocab_params = config['vocabulary']
        self.vocab = Vocabulary.from_params(vocab_params)

        self.iterator = DataIterator.from_params(
            Params({"type": "basic", "batch_size": batch_size})
        )
        self.iterator.index_with(self.vocab)

    def _replace_mask(self, s):
        return s.replace('[MASK]', ' [MASK] ')

    def iter_batches(self, sentences_or_sentence_pairs: Union[List[str], List[List[str]]], verbose=True):
        instances = []
        for sentence_or_sentence_pair in sentences_or_sentence_pairs:
            if isinstance(sentence_or_sentence_pair, list):
                assert len(sentence_or_sentence_pair) == 2
                tokens_candidates = self.tokenizer_and_candidate_generator.\
                        tokenize_and_generate_candidates(
                                self._replace_mask(sentence_or_sentence_pair[0]),
                                self._replace_mask(sentence_or_sentence_pair[1]))
            else:
                tokens_candidates = self.tokenizer_and_candidate_generator.\
                        tokenize_and_generate_candidates(self._replace_mask(sentence_or_sentence_pair))

            if verbose:
                print(self._replace_mask(sentence_or_sentence_pair))
                print(tokens_candidates['tokens'])

            if self.masking_strategy == 'full_mask':
                masked_indices = [index for index, token in enumerate(tokens_candidates['tokens'])
                      if token == '[MASK]']

                spans_to_mask = set([(i, i) for i in masked_indices])
                replace_candidates_with_mask_entity(
                        tokens_candidates['candidates'], spans_to_mask
                )

                for key in tokens_candidates['candidates'].keys():
                    for span_to_mask in spans_to_mask:
                        found = False
                        for span in tokens_candidates['candidates'][key]['candidate_spans']:
                            if tuple(span) == tuple(span_to_mask):
                                found = True
                        if not found:
                            tokens_candidates['candidates'][key]['candidate_spans'].append(list(span_to_mask))
                            tokens_candidates['candidates'][key]['candidate_entities'].append(['@@MASK@@'])
                            tokens_candidates['candidates'][key]['candidate_entity_priors'].append([1.0])
                            tokens_candidates['candidates'][key]['candidate_segment_ids'].append(0)
                            assert not isinstance(sentence_or_sentence_pair, list)


            fields = self.tokenizer_and_candidate_generator.\
                convert_tokens_candidates_to_fields(tokens_candidates)

            instances.append(Instance(fields))


        for batch in self.iterator(instances, num_epochs=1, shuffle=False):
            yield batch


processor = KnowBERTProcessor(root_path='/content/drive/My Drive/Gatech courses/Fall 2024 Courses/Natural Language Processing/NLP Project')

try:

    isear_df, conceptnet_df = processor.load_datasets(
        isear_path='Datasets/cleaned_full_isear_data_with_sentiment.csv',
        conceptnet_path='Knowledge_graph/conceptnet-assertions-5.7.0.csv.gz'
    )

    processor.initialize_knowbert()

    embeddings = processor.process_dataset(
        df=isear_df.head(10),
        conceptnet_df=conceptnet_df,
        save_path="knowbert_embeddings.npy"
    )

except Exception as e:
    logger.error(f"Error during processing: {str(e)}")
    raise



In [ ]:
# Feyi
isear_path='Datasets/cleaned_full_isear_data_with_sentiment.csv'
root_path='/content/drive/My Drive/Gatech courses/Fall 2024 Courses/Natural Language Processing/NLP Project'
full_isear_path = os.path.join(root_path, isear_path)
df = pd.read_csv(full_isear_path, nrows=1)
print(print(df['content'].tolist()))